In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch, csd, butter, lfilter, freqz
from speckit import lpsd
from multiprocessing import Pool
pool = Pool()

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Setting standard filter requirements.
order = 3
fs = 2
cutoff = 1e-3

b, a = butter_lowpass(cutoff, fs, order)
w, h = freqz(b, a, worN=8000)

plt.subplot(2, 1, 1)
plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')
plt.loglog(cutoff, 0.5*np.sqrt(2), 'ko')
plt.axvline(cutoff, color='k')
plt.xlim(0, 0.5*fs)
plt.title("Lowpass Filter Frequency Response")
plt.xlabel('Frequency [Hz]')
plt.grid()
plt.show()

N  = int(1e6)
x1 = np.random.rand(N)
x2 = butter_lowpass_filter(x1, cutoff, fs, order)

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')
ax.plot(x1, color='gray', label='Input signal (x1)')
ax.plot(x2, color='k', label='Filtered signal (x2)')
plt.show()

f1, ps1 = welch(x1, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='density', return_onesided=True, window='hann')
f2, ps2 = welch(x2, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='density', return_onesided=True, window='hann')
f3, csd12 = csd(x1, x2, fs=fs, nperseg=int(1e6/2), noverlap=int(1e6/4), scaling='density', return_onesided=True, window='hann')

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral densities (${\rm units}^2/{\rm Hz}$)')
ax.loglog(f1, ps1, label='PSD(x1)')
ax.loglog(f2, ps2, label='PSD(x2)')
ax.loglog(f3, np.abs(csd12), label='CSD(x1,x2)')
ax.legend()
plt.show()

In [ ]:
bmin = 1.0
order = -1
olap = 0.5
Lmin = int(7e3)

# About 4 seconds on Apple M4 Max
obj  = lpsd.ltf([x1,x2], fs=fs, olap=olap, bmin=bmin, Lmin=Lmin, order=order, pool=pool, win="hann")

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.set_xlabel('Fourier frequency (Hz)')
ax.set_ylabel(r'Spectral densities ${\rm units}^2/{\rm Hz}$')
ax.loglog(f1, ps1, label="PSD(x1) (Welch)")
ax.loglog(f2, ps2, label="PSD(x2) (Welch)")
ax.loglog(f3, np.abs(csd12), label='|CSD(x1,x2)| (Welch)')
ax.loglog(obj.f, obj.Gxx, label="PSD(x1) (spectools)", color='lightblue', ls='--')
ax.loglog(obj.f, obj.Gyy, label="LPSD(x2) (spectools)", color='yellow', ls='--')
ax.loglog(obj.f, np.abs(obj.Gxy), label="|CSD(x1,x2)| (spectools)", color='lightgreen', ls='--')
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax2, ax1) = obj.plot(which='bode', errors=True, sigma=3, dB=True)
ax1.set_ylim(-360,60)
plt.show()

In [ ]:
fig, ax = obj.plot(which='csd', errors=True)
plt.show()

In [ ]:
fig, ax = obj.plot(which='coh', errors=True, sigma=5,
                   c='darkred', lw=1.0, ls=':', label=r'$\gamma_{xy}^2$')
plt.show()